In [3]:
from __future__ import print_function
from random import randint
from tensorflow import keras
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation
import tensorflow as tf
import time
import math

In [4]:
batch_size = 256
num_classes = 10
epochs = 20

In [5]:
# train the model
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(Y_train.shape)
print(Y_test.shape)

(60000,)
(10000,)


In [6]:
def normalize(array,quanti=8):#normalize and quantize the weight, denormalize in feedfoward
    w=np.asarray(array)
    absmax=np.amax(np.abs(w))
    w = w/absmax
    quantScale=1/quanti/2
    for i in range(-quanti,quanti):
        j=i/quanti+quantScale
        w[np.abs(w-j)<=quantScale] = j
    return w

def ifp_normalize(array,levels=16):#normalize and quantize the input, denormalize in feedfoward
    w=np.asarray(array)
    absmax=np.amax(np.abs(w))
    w = w/absmax
    quantScale=1/levels/2
    for i in range(0,levels):
        j=i/levels+quantScale
        w[np.abs(w-j)<=quantScale] = j
    return w

In [7]:
x_train = X_train.reshape(60000, 784)
x_test = X_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train=ifp_normalize(x_train,16) * 4
x_test=ifp_normalize(x_test,16) * 4
num_classes = 10
y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [8]:
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend as K

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, c=1):
        self.c = c

    def __call__(self, p):
        return K.clip(p, -self.c, self.c)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'c': self.c}
layers=4
def propagation_loss(length=10*10e-4,loss_per_length=0.3):
    return 1/10**(length*loss_per_length/10)
def splitter_loss(loss=0.2):
    return 1/10**(loss/10)
def coupling_loss(layers=1):
    return 1/10**(layers*0.5/10)
def layer_loss(leaf = 256):
    s_n = math.log2(leaf)+3
    p_n = s_n+1
    c_n = 2
    return 1/leaf *1/2 * (splitter_loss())**(s_n-1) * propagation_loss()**(p_n-1) *coupling_loss()**c_n
use_bias = False
def create_model():
    minPDin=2e-7
    wq_lv=7
    model = Sequential(name='model1')
    retrain= False
    global layers
    global use_bias
    if retrain: 
        for i in range(layers-1):
            model.add(Dense(256, input_dim=784,activation=None, use_bias = use_bias, kernel_constraint = WeightClip(1)))
            model.add(ReLU(max_value=1/layer_loss(), negative_slope=0.0, threshold=minPDin/layer_loss()))
        model.add(Dense(10, activation='softmax',use_bias = use_bias, kernel_constraint = WeightClip(1)))
    #model.add(GaussianNoise(noise_para))
    else:
        for i in range(layers-1):
            model.add(Dense(256,input_dim=784,use_bias = use_bias, activation='relu'))
        model.add(Dense(10,use_bias = use_bias,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(), 
                  metrics=['accuracy'])
    return model
   

model = create_model()

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               200704    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2560      
Total params: 334,336
Trainable params: 334,336
Non-trainable params: 0
_________________________________________________________________


In [371]:
# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s 26us/sample - loss: 0.3209 - acc: 0.9059 - val_loss: 0.1470 - val_acc: 0.9565
Epoch 2/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.1148 - acc: 0.9652 - val_loss: 0.1136 - val_acc: 0.9645
Epoch 3/20
60000/60000 [==============================] - 1s 18us/sample - loss: 0.0750 - acc: 0.9769 - val_loss: 0.0860 - val_acc: 0.9718
Epoch 4/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.0531 - acc: 0.9833 - val_loss: 0.0795 - val_acc: 0.9742
Epoch 5/20
60000/60000 [==============================] - 1s 18us/sample - loss: 0.0393 - acc: 0.9874 - val_loss: 0.0728 - val_acc: 0.9771
Epoch 6/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.0332 - acc: 0.9894 - val_loss: 0.0691 - val_acc: 0.9783
Epoch 7/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.0271 - acc: 0.9910 - val_loss: 0.

In [372]:
#model.save_weights('my_model_weights.h5')

In [10]:
model2 = Sequential()
for i in range(layers-1):
    model2.add(Dense(256,input_dim=784,use_bias = use_bias, activation='relu'))
model2.add(Dense(10, use_bias = use_bias,activation='softmax'))

model2.load_weights('mnist_DACTL_weights.h5')

0.31361568
0.4523125
0.50465083
0.32831943


In [34]:

def feedforward2(layer,inputs,weights,bias,variation,activate_relu=1,leaf=256,spread_branch=1/256):
    # Convert signal from electrical domain to optical domain by TL in former layer\
    inputs[inputs>4]=4
    Splitter_Variation=0.05
    tl_variation=0.05
    inputs = inputs * np.random.uniform(1-tl_variation,1+tl_variation,inputs.shape) * coupling_loss() 
    copy_num = weights.shape[1]
    spliter_count = math.log2(leaf) + 3
    #print(inputs.shape[0])
    # Distribute inputs by splitter trees
    random_input = np.repeat(inputs,copy_num,0).reshape(inputs.shape[0],copy_num) \
    * spliter_forest(inputs.shape[0],copy_num)
    # Add variation to splitters for weight calculation 
    weights_random = weights * np.random.uniform(1-Splitter_Variation,1+Splitter_Variation,weights.shape)
    
    # Do the weight calculation 
    result = random_input * weights_random * propagation_loss() * splitter_loss()
    
    # Convert signal from optical domain to electrical domain by PD with 1:1 splitter for PD selection
    result = result * 0.5 * splitter_loss()* propagation_loss()\
    *np.random.uniform(1-Splitter_Variation,1+Splitter_Variation,result.shape) * coupling_loss() 
#     if activate_relu!=1:
#         print(np.amax(result))
    # Set lower bound for input of PD
    
    result[np.abs(result)<2e-7]=0
    
    # Accumulation
    result = np.sum (result,0)
    # Amplify the result with corresponding coeffient to obtaint correct output of this layer
  
    # If it is not the last layer, implement ReLU and set upper bound for output at 1mW
    global use_bias
    if use_bias!=False:
        for i in range(0,layer+1):
            bias =  bias * 255/256 * 1/leaf *1/2 *\
                    splitter_loss() ** spliter_count * propagation_loss() ** spliter_count * coupling_loss() ** 2
    #     print('o2 bias:',end='')
    #     print(bias)
    #     global bias_coe
    #     bias *= bias_coe
    output = result  + bias
    global layers
    if layer!=layers-1:
        output=relu(output)
#     print('o2 output:',end='')
    #print(output)
    return output


def relu(output):
  nextinput = np.zeros((output.shape[0]))
  for i in range(nextinput.shape[0]):
    nextinput[i] = max(0,output[i])
  return nextinput
#@jit

In [35]:

def network(x_test,y_test,variation):#x_test is 1*784, one 28*28 image
    weights=[]
    bias=[]
    wmax=np.zeros(layers)
    global use_bias
    for i in range(0,layers):
        weights.append(np.array(model2.layers[i].get_weights()[0]))
        if use_bias == False:
            bias.append(0)
        else:
            bias.append(np.array(model2.layers[i].get_weights()[1]))
        wmax[i]=np.amax(np.abs(weights[i]))
    o2 = feedforward2(0,x_test,normalize(weights[0]),bias[0],wmax[0],variation)
#     o2 = amplifier(o2,1/layer_loss()*wmax[0],0.05)
    o2 = feedforward2(1,o2,normalize(weights[1]),bias[1],wmax[1],variation)
    o2 = amplifier(o2,1/layer_loss()**2,0.05)
    o2 = feedforward2(2,o2,normalize(weights[2]),bias[2],wmax[2],variation)
#     o2 = amplifier(o2,1/layer_loss()*wmax[2],0.05)
    o2 = feedforward2(3,o2,normalize(weights[3]),bias[3],wmax[3],variation)
    pred_out = np.zeros((1,o2.shape[0]))
    pred_out[0] = o2
    correct_out = np.zeros((1,y_test.shape[0]))
    correct_out[0] = y_test
    #print(np.amax(pred_out))
    return (np.argmax(pred_out) == np.argmax(correct_out))


def spliter_tree(root_value=1,leaf=256,variation=0.05):
    height=math.ceil(math.log2(leaf))+1
    arr=np.ones(int('1'*height, 2))
    for i in range(1,height):
        left = np.random.uniform(1-variation,1+variation,2**(i-1))*1/2
        right = 1-left
        branch = np.concatenate((left,right)).flatten('F')*propagation_loss()*splitter_loss()
        arr[2**i-1:2**(i+1)-1]=np.repeat(arr[2**(i-1)-1:2**i-1],2)*branch
    return arr[2**(height-1)-1:2**height-1]


def spliter_forest(input_num=784,grove_len=256,leaf=256,spliter_variation=0.05,amplifier_variation=0.05,spread_branch=1/256):
    arr=np.zeros((input_num,grove_len))
    if grove_len<256:
        leaf=64
    for i in range(0,input_num):
        arr[i]=spliter_tree(1,leaf)[:grove_len]
    return arr 

def amplifier(x, multiple, amp_vari=0.05):
    delta_in= x * np.random.uniform(-amp_vari,amp_vari)
    x_out = x * multiple + multiple * 6.25 * delta_in*10**(1/2)
    return x_out

In [39]:

def main(logname='MNIST_10mWTL_NoAmp.txt'):
    log=open(logname,'w')
    log.write(time.strftime("%Y-%m-%d %H:%M:%S\n", time.localtime()))
    log.close()
    result=0
    j=5
    iteration = 100
    for k in range(iteration):
        correct_test = 0
        for i in range(0,x_test.shape[0]):
#             if True:
            if (network(x_test[i],y_test[i],0.01*j)):
                correct_test += 1
            if (i+1)%10==0: # to test the function
               print ('inference with variation %f current accuracy at %i:'%(0.01*j,i+1),correct_test/(i+1))
        log = open(logname,'a')
        log.write('The accuracy of inference %d is %f\n'%(k,correct_test/10000))
        result += correct_test/10000
    log=open(logname,'a')
    log.write('The average accuracy is %f\n'%(result/iteration))
    log.close()
    
main()

inference with variation 0.050000 current accuracy at 10: 1.0
inference with variation 0.050000 current accuracy at 20: 1.0
inference with variation 0.050000 current accuracy at 30: 1.0
inference with variation 0.050000 current accuracy at 40: 1.0
inference with variation 0.050000 current accuracy at 50: 1.0
inference with variation 0.050000 current accuracy at 60: 1.0
inference with variation 0.050000 current accuracy at 70: 1.0
inference with variation 0.050000 current accuracy at 80: 1.0
inference with variation 0.050000 current accuracy at 90: 1.0
inference with variation 0.050000 current accuracy at 100: 1.0
inference with variation 0.050000 current accuracy at 110: 1.0
inference with variation 0.050000 current accuracy at 120: 0.9833333333333333
inference with variation 0.050000 current accuracy at 130: 0.9846153846153847


KeyboardInterrupt: 